# Google's Landmark Retrieval Challenge

**Goal**: Given over a million images of 15K classes, you need to classify them correctly as they lack labels. Such that given a test image, it should be able to retrieve images that belong to similar class.

[Kaggle Challenge](https://www.kaggle.com/c/landmark-retrieval-challenge)

In [1]:
import pandas as pd
import urllib
import shutil
import cv2
import os
import numpy as np
from keras.preprocessing import image     

C:\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#STEP #1: Download images
train_data = pd.read_csv("index.csv")
test_data = pd.read_csv("test.csv")

#print("Shape of training set = ", train_data.shape)
#print("Shape of testing set = ", test_data.shape)

def download_images(csv_file, outputpath, img_file_type):
    if not os.path.isdir(outputpath):
        os.makedirs(outputpath, exist_ok=True)
    
    count = 0
    for index, row in csv_file.iterrows():
        img_id = row['id']
        img_url = row['url']
        img_title = outputpath + '/' +img_id + '.' + img_file_type
        if index < 20:
            try:
                urllib.request.urlretrieve(img_url,img_title) # saves local copy of images
                count += 1
            except:
                print(img_id + "<-- couldn't retrieve IMAGE for this id!!")
        else:
            print("Downloaded ==> " + str(count) + " files!") 
            return

In [3]:
#download_images( train_data, 'train','jpg')
#download_images(test_data, 'test','jpg')

In [4]:
def load_dataset(inputpath):
    list_of_tensors = []
    file_names = []
    for dirpath, dirnames, filenames in os.walk(inputpath):
        #print("Files in this '" + inputpath + "' dir is " + str(len(filenames)))
        for filename in filenames:
            img_path = os.path.join(dirpath,filename)
            img = image.load_img(img_path, target_size=(224, 224))#,grayscale=True) #rescaling images to 224x224
            x = image.img_to_array(img)
            list_of_tensors.append(np.expand_dims(x, axis=0))
        file_names = filenames
    return np.vstack(list_of_tensors), file_names

train_tensors, train_files = load_dataset('train')
test_tensors, test_files = load_dataset('test')

train_tensors = train_tensors.astype('float32')/255 # rescale to [0,1] instead of [0,255] for each pixel
test_tensors= test_tensors.astype('float32')/255

#Train and validation split
(train_tensors, valid_tensors) = train_tensors[5:], train_tensors[:5]
(train_files, valid_files) = train_files[5:], train_files[:5]

print("Training set: ", train_tensors.shape)
print("Validation set: ", valid_tensors.shape)
print("Testing set: ", test_tensors.shape)

Training set:  (16, 224, 224, 3)
Validation set:  (5, 224, 224, 3)
Testing set:  (21, 224, 224, 3)


In [5]:
print(train_files)
print(test_files)

['4667f19f45f6a155.jpg', '5e1e25cdfca4fdb0.jpg', '6648383c7b3a438c.jpg', '890ddd72ea63812c.jpg', 'b03b500b28655072.jpg', 'b09ea096f4daa42e.jpg', 'bbecba89db6a9bec.jpg', 'cd7ca283cf77d133.jpg', 'd485d9f770e40453.jpg', 'de5ae5ff9dbe21eb.jpg', 'e7cdfba9ec7e9dd5.jpg', 'eb514a54707ac0bc.jpg', 'eiffeltower.jpg', 'f0367bb0b4c2d526.jpg', 'f3ba87e473810669.jpg', 'fe2ffda14b81dbbc.jpg']
['000088da12d664db.jpg', '0001623c6d808702.jpg', '0001bbb682d45002.jpg', '0002362830cfe3a3.jpg', '000270c9100de789.jpg', '0002b0fab5d3ccc4.jpg', '000396be3c24830a.jpg', '000506dc6ab3a40e.jpg', '0005292fc4b005a3.jpg', '0005456a82264bc8.jpg', '00055cf2bfb5594a.jpg', '000664eed4a70821.jpg', '0006aea5b6f4eaaa.jpg', '0006bbfa00dd6c0f.jpg', '0008aee1c0abed9d.jpg', '0008de5f3c25d563.jpg', '00094466c9f054f4.jpg', '0009f09a69405693.jpg', '000a7f4a25af7558.jpg', '000a9180e4dc8705.jpg', 'eiffeltower_angle2.jpg']


In [6]:
# get feature set from ImageNet using Keras model - ResNet50
from keras import applications
model = applications.resnet50.ResNet50(include_top=False, weights='imagenet', pooling='avg')

# extract the features
train_targets = model.predict(train_tensors)#[0]
valid_targets = model.predict(valid_tensors)#[0]

# convert from Numpy to a list of values
train_targets_arr = np.char.mod('%f', train_targets)
valid_targets_arr = np.char.mod('%f', valid_targets)
#features_dict = {"id": metadata['id'], "features": ','.join(features_arr)}

In [15]:
print(train_targets_arr.shape)
print(valid_targets_arr.shape)
print(train_targets_arr.shape[1])

# from sklearn.manifold import TSNE
# train_targets_dr = TSNE(n_components=3).fit_transform(train_targets)
# valid_targets_dr = TSNE(n_components=3).fit_transform(valid_targets)
# print(train_targets_dr.shape, valid_targets_dr.shape)

(16, 2048)
(5, 2048)
2048


In [19]:
#Implementing basic CNN from scratch
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

numOfLabels = (train_targets_arr.shape[1]) # based on the dataset description  --- NOT TOO SURE HERE!!! I thought 15K should be the value

cnn = Sequential()
cnn.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(224,224,3)))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(Dropout(0.3))
cnn.add(Flatten())
cnn.add(Dense(500, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(numOfLabels, activation='softmax'))

cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 224, 224, 16)      208       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 56, 56, 64)        8256      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 28, 28, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 28, 28, 64)        0         
__________

In [28]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [29]:
# Step 7: Train the model
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='weights.best.from_scratch.hdf5', verbose=1, save_best_only=True)

cnn.fit(train_tensors, train_targets_arr, validation_data=(valid_tensors, valid_targets_arr), epochs=10, batch_size=1, callbacks=[checkpointer], verbose=1)


Train on 16 samples, validate on 5 samples
Epoch 1/10
15/16 [===========================>..] - ETA: 1s - loss: 0.4975 - acc: 0.7275 

In [30]:
# load the weights that yielded the best validation accuracy
cnn.load_weights('weights.best.from_scratch.hdf5')

In [31]:
from skimage import measure
test_labels_predicted = cnn.predict(test_tensors)
# counter = 0
# for test_img in test_tensors:
#     #predicted = cnn.predict(test_tensors)
#     ssim = []
#     for train_img in train_tensors:
#         ssim.append(measure.compare_ssim(test_img, train_img, multichannel=True,full=True))
#     #print("Label predicted for test_img", test_files[counter], "=", max(predicted[counter]), " @ column =", np.argmax(predicted[counter]) )
#     print("SSIM for test_img #", test_files[counter], " & train_img",train_files[ssim.index(max(ssim))], "=", max(ssim)[0])
#     counter +=1


In [32]:
test_labels_predicted.shape

(21, 2048)

In [33]:
# Step 8: Calculate accuracy
score = cnn.evaluate(test_tensors, test_labels_predicted, verbose=0)
accuracy = score[1] *100

print("Test accuracy = ", accuracy, "%")

Test accuracy =  0.0 %
